In [2]:
import cv2
import numpy as np

def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text) + '1111111111111110'  # 结束标志

def binary_to_text(binary):
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join(chr(int(char, 2)) for char in chars if int(char, 2) != 0)

def hide_text(cover_path, text, stego_path):
    cover = cv2.imread(cover_path)
    binary_text = text_to_binary(text)
    data_index = 0
    data_length = len(binary_text)
    
    for row in cover:
        for pixel in row:
            for channel in range(3):
                if data_index < data_length:
                    pixel[channel] = (pixel[channel] & 0xFE) | int(binary_text[data_index])
                    data_index += 1
                else:
                    break
    cv2.imwrite(stego_path, cover)
    print("Text hidden successfully!")

def extract_text(stego_path):
    stego = cv2.imread(stego_path)
    binary_text = ''
    
    for row in stego:
        for pixel in row:
            for channel in range(3):
                binary_text += str(pixel[channel] & 1)
                if binary_text[-16:] == '1111111111111110':  # 结束标志
                    return binary_to_text(binary_text[:-16])
    return "Message not found"

# 示例用法
hide_text('./dataset/cover.jpg', 'Hello, Steganography!', './results/stego.png')
print(extract_text('./results/stego.png'))

Text hidden successfully!
Hello, Steganography!


In [1]:
import math
from PIL import Image
from skimage.metrics import structural_similarity as ssim
import numpy as np


def psnr(img1, img2):
    """计算两个图像之间的 PSNR"""
    mse = np.mean((np.array(img1) - np.array(img2)) ** 2)
    if mse == 0:
        return float('inf')  # 完全相同的图像
    max_pixel = 255.0
    return 20 * math.log10(max_pixel / math.sqrt(mse))

def compare_images(cover_path, stego_path):
    """对比原始图像和隐藏图像之间的差异"""
    
    cover_image = Image.open(cover_path)
    stego_image = Image.open(stego_path)

    # 计算 PSNR
    psnr_value = psnr(cover_image, stego_image)
    print(f"PSNR between cover and stego image: {psnr_value} dB")

    # 计算 SSIM
    cover_array = np.array(cover_image.convert('RGB'))
    stego_array = np.array(stego_image.convert('RGB'))
    
    ssim_value = ssim(cover_array, stego_array, multichannel=True, win_size=3)
    print(f"SSIM between cover and stego image: {ssim_value}")

# 示例用法
cover_path = "./dataset/cover.jpg"  # 载体图像路径
stego_path = "./results/stego.png"  # 隐藏图像路径

# 对比原始图像和隐藏图像
compare_images(cover_path, stego_path)


PSNR between cover and stego image: 101.34652215750518 dB
SSIM between cover and stego image: 0.9999999923938419
